# RunPod 聊天模型

开始使用 RunPod 聊天模型。

## 概述

本指南介绍了如何使用 LangChain 的 `ChatRunPod` 类与托管在 [RunPod Serverless](https://www.runpod.io/serverless-gpu) 上的聊天模型进行交互。

## 设置

1.  **安装包：**
    ```bash
    pip install -qU langchain-runpod
    ```
2.  **部署聊天模型端点：** 按照 [RunPod 提供商指南](/docs/integrations/providers/runpod#setup) 中的设置步骤，在 RunPod Serverless 上部署兼容的聊天模型端点并获取其 Endpoint ID。
3.  **设置环境变量：** 确保设置了 `RUNPOD_API_KEY` 和 `RUNPOD_ENDPOINT_ID`（或特定的 `RUNPOD_CHAT_ENDPOINT_ID`）。

In [ ]:
import getpass
import os

# Make sure environment variables are set (or pass them directly to ChatRunPod)
if "RUNPOD_API_KEY" not in os.environ:
    os.environ["RUNPOD_API_KEY"] = getpass.getpass("Enter your RunPod API Key: ")

if "RUNPOD_ENDPOINT_ID" not in os.environ:
    os.environ["RUNPOD_ENDPOINT_ID"] = input(
        "Enter your RunPod Endpoint ID (used if RUNPOD_CHAT_ENDPOINT_ID is not set): "
    )

# Optionally use a different endpoint ID specifically for chat models
# if "RUNPOD_CHAT_ENDPOINT_ID" not in os.environ:
#     os.environ["RUNPOD_CHAT_ENDPOINT_ID"] = input("Enter your RunPod Chat Endpoint ID (Optional): ")

chat_endpoint_id = os.environ.get(
    "RUNPOD_CHAT_ENDPOINT_ID", os.environ.get("RUNPOD_ENDPOINT_ID")
)
if not chat_endpoint_id:
    raise ValueError(
        "No RunPod Endpoint ID found. Please set RUNPOD_ENDPOINT_ID or RUNPOD_CHAT_ENDPOINT_ID."
    )

## 实例化

初始化 `ChatRunPod` 类。您可以通过 `model_kwargs` 传递模型专属参数，并配置轮询行为。

In [ ]:
from langchain_runpod import ChatRunPod

chat = ChatRunPod(
    runpod_endpoint_id=chat_endpoint_id,  # Specify the correct endpoint ID
    model_kwargs={
        "max_new_tokens": 512,
        "temperature": 0.7,
        "top_p": 0.9,
        # Add other parameters supported by your endpoint handler
    },
    # Optional: Adjust polling
    # poll_interval=0.2,
    # max_polling_attempts=150
)

## 调用

使用标准的 LangChain `.invoke()` 和 `.ainvoke()` 方法来调用模型。还支持通过 `.stream()` 和 `.astream()` 进行流式调用（通过轮询 RunPod `/stream` 端点模拟）。

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="What is the RunPod Serverless API flow?"),
]

# Invoke (Sync)
try:
    response = chat.invoke(messages)
    print("--- Sync Invoke Response ---")
    print(response.content)
except Exception as e:
    print(
        f"Error invoking Chat Model: {e}. Ensure endpoint ID/API key are correct and endpoint is active/compatible."
    )

# Stream (Sync, simulated via polling /stream)
print("\n--- Sync Stream Response ---")
try:
    for chunk in chat.stream(messages):
        print(chunk.content, end="", flush=True)
    print()  # Newline
except Exception as e:
    print(
        f"\nError streaming Chat Model: {e}. Ensure endpoint handler supports streaming output format."
    )

### Async Usage

# AInvoke (Async)
try:
    async_response = await chat.ainvoke(messages)
    print("--- Async Invoke Response ---")
    print(async_response.content)
except Exception as e:
    print(f"Error invoking Chat Model asynchronously: {e}.")

# AStream (Async)
print("\n--- Async Stream Response ---")
try:
    async for chunk in chat.astream(messages):
        print(chunk.content, end="", flush=True)
    print()  # Newline
except Exception as e:
    print(
        f"\nError streaming Chat Model asynchronously: {e}. Ensure endpoint handler supports streaming output format.\n"
    )

## 链式调用

聊天模型可以与 LangChain Expression Language (LCEL) 链无缝集成。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

parser = StrOutputParser()

chain = prompt | chat | parser

try:
    chain_response = chain.invoke(
        {"input": "Explain the concept of serverless computing in simple terms."}
    )
    print("--- Chain Response ---")
    print(chain_response)
except Exception as e:
    print(f"Error running chain: {e}")


# Async chain
try:
    async_chain_response = await chain.ainvoke(
        {"input": "What are the benefits of using RunPod for AI/ML workloads?"}
    )
    print("--- Async Chain Response ---")
    print(async_chain_response)
except Exception as e:
    print(f"Error running async chain: {e}")

## 模型功能（取决于端点）

高级功能的可用性**严重**依赖于您 RunPod 端点处理程序的具体实现。`ChatRunPod` 集成提供了基本框架，但处理程序必须支持底层功能。

| 功能                                                       | 集成支持 | 是否取决于端点？ | 备注                                                                                                                                                                       |
| :--------------------------------------------------------- | :------: | :--------------: | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| [工具调用](/docs/how_to/tool_calling)                    |    ❌    |        ✅        | 要求处理程序处理工具定义并返回工具调用（例如，OpenAI 格式）。集成需要解析逻辑。                                                                                             |
| [结构化输出](/docs/how_to/structured_output)              |    ❌    |        ✅        | 要求处理程序支持强制结构化输出（JSON 模式、函数调用）。集成需要解析逻辑。                                                                                                 |
| JSON 模式                                                  |    ❌    |        ✅        | 要求处理程序接受 `json_mode` 参数（或类似参数）并保证 JSON 输出。                                                                                                         |
| [图像输入](/docs/how_to/multimodal_inputs)                |    ❌    |        ✅        | 要求多模态处理程序接受图像数据（例如，base64）。集成不支持多模态消息。                                                                                                    |
| 音频输入                                                   |    ❌    |        ✅        | 要求处理程序接受音频数据。集成不支持音频消息。                                                                                                                          |
| 视频输入                                                   |    ❌    |        ✅        | 要求处理程序接受视频数据。集成不支持视频消息。                                                                                                                          |
| [Token 流式输出](/docs/how_to/chat_streaming)             |  ✅ (模拟) |        ✅        | 轮询 `/stream`。要求处理程序在状态响应中填充 `stream` 列表，其中包含 token 块（例如，`[{"output": "token"}]`）。真正的低延迟流式输出并非内置。                           |
| 原生异步                                                   |    ✅    |        ✅        | 已实现核心的 `ainvoke`/`astream`。依赖于端点处理程序的性能。                                                                                                             |
| [Token 使用量](/docs/how_to/chat_token_usage_tracking)    |    ❌    |        ✅        | 要求处理程序在最终响应中返回 `prompt_tokens`、`completion_tokens`。集成目前不解析此项。                                                                                 |
| [Logprobs](/docs/how_to/logprobs)                          |    ❌    |        ✅        | 要求处理程序返回 log probabilities。集成目前不解析此项。                                                                                                                 |

**关键要点：** 如果端点遵循 RunPod API 基本约定，则标准的聊天调用和模拟流式传输将正常工作。高级功能需要特定的处理程序实现，并可能需要扩展或自定义此集成包。

## API 参考

有关 `ChatRunPod` 类、参数和方法的详细文档，请参阅源代码或生成的 API 参考（如果可用）。

源代码链接：[https://github.com/runpod/langchain-runpod/blob/main/langchain_runpod/chat_models.py](https://github.com/runpod/langchain-runpod/blob/main/langchain_runpod/chat_models.py)